In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle

from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [27]:
train_xdata = np.array([np.reshape(x, (28*28)) for x in mnist.train.images])        #image files(channel = 1)
test_xdata = np.array([np.reshape(x, (28*28)) for x in mnist.test.images])
train_labels = mnist.train.labels
test_labels = mnist.test.labels

x = tf.placeholder(tf.float32, [None,784])                # 입력 X, 28*28크기의 이미지이고 채널은 1개
y = tf.placeholder(tf.float32, [None,10])
keep_prob = tf.placeholder(tf.float32)          #drop-out 되지 않을 확률 값을 넣기 위한 placeholder

x_data = tf.reshape(x, [-1,28,28,1])           # x 를 28*28*1 크기로 차원을 변경,  2d -> 3d

In [28]:
#바이어스를 0.1로 초기화하는 함수.
def bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#표준편차 0.1을 weight로 가지는 난수로 초기화하는 함수.
def weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


#stride=1,  padding=0으로 하는 convolution layer를 만드는 함수
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 2x2 max pooling layer
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 1st convolution layer 

In [29]:
#bias, weight
W_conv1 = weight([5, 5, 1, 32])        # color channel = 1, filter=32
b_conv1 = bias([32])

# convolution layer + max pooling layer
layer_conv1 = tf.nn.relu(conv2d(x_data, W_conv1) + b_conv1)
layer_pool1 = max_pool(layer_conv1)          

In [30]:
print(x_data.get_shape())
print(layer_conv1.get_shape())
print(layer_pool1.get_shape())
#SAME padding이므로 conv2d로는 차원이 변경되지 않고 max_pool에서 stride에 따라 차원이 반으로 줄어든다.

(?, 28, 28, 1)
(?, 28, 28, 32)
(?, 14, 14, 32)


# 2nd convolution layer

In [31]:
#bias, weight
W_conv2 = weight([5, 5, 32, 64])           #color channel = 32 (input), filter = 64
b_conv2 = bias([64])

# convolution layer + max pooling layer
layer_conv2 = tf.nn.relu(conv2d(layer_pool1, W_conv2) + b_conv2)
layer_pool2 = max_pool(layer_conv2)

In [32]:
print(layer_conv2.get_shape())     # 차원은 그대로(padding=same)
print(layer_pool2.get_shape())    # max_pool에서 stride에 따라 차원이 반으로 줄어든다.

(?, 14, 14, 64)
(?, 7, 7, 64)


# Fully connected layer

In [33]:
#마지막 단계인 softmax layer에 연결하기 위해 완전연결 레이어를 추가. 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

#바로 전 단계 conv2d layer 결과를 다시 1차원 tensor로 변환하여 relu 활성화 함수에 전달.
layer_pool2_flat = tf.reshape(layer_pool2, [-1, 7*7*64])    
layer_fc1 = tf.nn.relu(tf.matmul(layer_pool2_flat, W_fc1) + b_fc1)

In [34]:
#drop out layer
layer_fc1_drop = tf.nn.dropout(layer_fc1, keep_prob)    

#마지막 단계인 softmax layer
W_fc2 = weight([1024, 10])
b_fc2 = bias([10])

y_data=tf.nn.softmax(tf.matmul(layer_fc1_drop, W_fc2) + b_fc2)

In [35]:
# cross entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_data),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_data,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [37]:
# initializing
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

train_epoch=1000
batch_size=100
for i in range(train_epoch):
    batch = mnist.train.next_batch(batch_size)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
                x:batch[0], y: batch[1], keep_prob: 1.0})
    sess.run(train_step,feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
print("accuracy %g"% sess.run(
        accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

accuracy 0.9867
